## Question: Can text accumulation be modeled as a dynamical system?

There are several inferential leaps in the original paper that "feel right" but are strictly empirical observations. For example, defining higher $n$-legomena counts in terms of $n$th partial derivatives of the type count formula. This was inferred by looking at the equations and checked against data. But what is the intuition behind it? How do hapax counts relate to the first derivative of types? Can this be motivated in plain English, by modeling text accumulation as a dynamical system, described in the language of differential equations, whose solutions are those formulae described in the paper?

In [1]:
# bloody dependencies
import matplotlib.pyplot as plt
from nltk.corpus import gutenberg
import numpy as np
import pandas as pd

# custom classes
from legomena import Corpus

### Choosing a Book

We're going to do something a little different this time. Let's take a small example to be able to work with hands-on. For fun, let's take the first paragraph of Moby Dick: Call me Ishmael...

In [2]:
# moby dick
words = gutenberg.words("melville-moby_dick.txt")

# play example: paragraph 1
words = list(words)[4712:4940]
print(words[:3])

['Call', 'me', 'Ishmael']


## Text Accumulation

Let's simulate the accumulation of text by literally building a corpus of each snapshot of the book's opening: the WFD of the first word, the first two words, the first three words, etc.

In [5]:
# list of corpuses
corpi = [ Corpus(words[:n]) for n in range(len(words)) ]
total = corpi[-1]

# first three words
corpi[3].WFD

,type,freq
rank,,
1,Call,1
2,me,1
3,Ishmael,1


In [43]:
# k vectors
L = len(total.k)
N = sum(total.k)
kvecs = []
for corpus in corpi:
    kvec = corpus.k
    kvec.resize(L)
    kvec[0] = N - kvec.sum()
    kvecs.append(kvec)
kvecs = np.array(kvecs)
display(kvecs[-15:,:])

# delta vectors
deltas = kvecs[1:,:] - kvecs[:-1, :]
display(deltas[-15:,:])

# choice indexes
idx = [ np.argwhere(delta<0)[0][0] for delta in deltas ]
"".join([str(i) for i in idx])

array([[  9., 111.,   9.,   2.,   7.,   2.,   0.,   2.,   1.,   2.,   0.],
       [  9., 110.,  10.,   2.,   7.,   2.,   0.,   2.,   1.,   2.,   0.],
       [  8., 111.,  10.,   2.,   7.,   2.,   0.,   2.,   1.,   2.,   0.],
       [  8., 111.,  10.,   2.,   7.,   2.,   0.,   2.,   1.,   1.,   1.],
       [  7., 112.,  10.,   2.,   7.,   2.,   0.,   2.,   1.,   1.,   1.],
       [  6., 113.,  10.,   2.,   7.,   2.,   0.,   2.,   1.,   1.,   1.],
       [  5., 114.,  10.,   2.,   7.,   2.,   0.,   2.,   1.,   1.,   1.],
       [  5., 114.,  10.,   2.,   7.,   2.,   0.,   2.,   0.,   2.,   1.],
       [  4., 115.,  10.,   2.,   7.,   2.,   0.,   2.,   0.,   2.,   1.],
       [  3., 116.,  10.,   2.,   7.,   2.,   0.,   2.,   0.,   2.,   1.],
       [  2., 117.,  10.,   2.,   7.,   2.,   0.,   2.,   0.,   2.,   1.],
       [  2., 117.,  10.,   2.,   7.,   2.,   0.,   2.,   0.,   1.,   2.],
       [  1., 118.,  10.,   2.,   7.,   2.,   0.,   2.,   0.,   1.,   2.],
       [  0., 119.,  10.,

array([[ 0., -1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0., -1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [-1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  1.],
       [-1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [-1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [-1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  1.,  0.],
       [-1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [-1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [-1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  1.],
       [-1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [-1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0., -1.,  1.,  0.,  0.,  0.,  0.,  0.]])

'00000000000001000000000000001001001000011000001010010201002020302030000140000120200110114110000033005020050240220000003240103000103000060550000001206602002130001703023000604040000000038004705031020600003700030080110900080009004'

We have here the empirical data embodying the phenomenon of text accumulation. At any given time, you have a bag of words $W$ describable by a word frequency distribution $\langle W \rangle$, itself describable as a distribution over those counts $\langle \langle W \rangle \rangle$. In the example above, we have $M=228$ tokens of $N=145$ types, $H=119$ of which are hapaxes, that is, tokens that appear exactly once. Sticking with our established notation, we use a zero-based vector $\hat{k}(x)$ to denote the evolution of $\langle \langle W \rangle \rangle$ as a function of sample size $x \in [0,1]$. Thus, $M=\sum{nk_n(x)}$ and $N=\sum{k_n(x)}$ for all $x$. Note that the zeroth component allows us to track the "number of types _not_ sampled". Therefore, if we're being particular about the summary indexes, then $N=\sum_{n=0}^\infty k_n(x)$ and $E(x)=\sum_{n=1}^\infty k_n(x) = N - k_0(x)$.

This starting notation in hand, what happens when we're in state $\hat{k}$ and draw one new token? For starters, _something_ must change. Either an unknown word is drawn which becomes a hapax, so $k_0$ decreases by one and $k_1$ increases by one. Or a known word is drawn. If this known word repeated in the text $n$ times, then now it repeats $n+1$ times, so $k_n$ decreases by one and $k_{n+1}$ increases by one. What we've described here is a Markov process describable as state $\hat{k}$ with legal transitions $n \to n+1$, "choose a token which repeated $n$ times and now thus repeats $n+1$ times" for $n \ge 0$. This dramatically reduces the dimension of the problem, by realizing this is the _only_ step transition possible.

Denote the probability of each transition by $\delta_n(x) = Pr(n \to n+1)$ at sample size $x$. And you now have expressions for the _change in $\hat{k}$_ over time:

$$
\hat{k}'(x) = \langle k_0'(x), k_1'(x), k_2'(x), k_3'(x), ...\rangle \\
k_0'(x) = -\delta_0(x) \\
k_n'(x) = -\delta_{n-1} + \delta_n(x)
$$

So, what _is_ the probability of each transition? Well, the total number of _tokens_ in the system is $M$. At sample fraction $x$ that gives $xM$ tokens drawn and $(1-x)M$ tokens undrawn. The number of tokens in "bin" $n$ at time $x$ is $nk_n(x)$ for $n>0$ and $(1-x)M$ for $n=0$, so the probability of drawing a token from bin $n$ is $\frac{1}{M}nk_n(x)$ for $n>0$ and $1-x$ for $n=0$.

In [54]:
# probability of transition
M = total.M
m = 99
nvec = np.array(range(L))
kvec = kvecs[m]
delta = nvec * kvec / M
delta[0] = 1-m/M

# display
print("Pr(0->1) =", delta[0])
print("Pr(n->n+1) =", delta)

Pr(0->1) = 0.5638766519823788
Pr(n->n+1) = [0.56387665 0.2246696  0.10572687 0.02643172 0.03524229 0.04405286
 0.         0.         0.         0.         0.        ]


1.0